# Experiments with storing ragged string arrays

In sgkit we have the limitation of not being able to store "ragged" arrays of strings very efficiently. The problem is discussed in https://github.com/pystatgen/sgkit/issues/634, in the context of storing variant alleles. Each variant may have an unknown and unbounded number of alleles, and storing such a dataset in Zarr is inefficient and wasteful.

This notebook has some experiments to explore this problem, using various storage and query approaches.

One of the ways that this data is used is for filtering, so it is useful to look at some example queries that might be  run against ragged data. We do this for variant alleles, but an actual use case for filtering VEP annotations can be found in https://github.com/pystatgen/sgkit/issues/1059, and is very similar.

The following queries are covered
1. Filter variants by a scalar variable. E.g. variants that have a certain position.
2. Filter variants by a string that occurs in the array. E.g. variants that have an "A" allele.
3. Filter variants by a string that is contained in strings in the array. E.g. variants that have allele containing a "T" base.
4. Filter variants by the size of the array. E.g. variants that have no alt alleles.

Query 1 is a simple baseline query that doesn't involve ragged arrays (or strings).

## Pandas

Regular Pandas support ragged string arrays. Here is an example for all of the queries.

In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame({'id':[1,2,3,4], 'alleles':[['A','C'],['G','A'],['T'],['G','GA','GAC']]})
df

,id,alleles
0,1,"[A, C]"
1,2,"[G, A]"
2,3,[T]
3,4,"[G, GA, GAC]"


In [3]:
# 1. variants that have a certain id.
df[df.id > 1]

,id,alleles
1,2,"[G, A]"
2,3,[T]
3,4,"[G, GA, GAC]"


In [4]:
# 2. variants that have an "A" allele (from https://stackoverflow.com/a/66095374)
idx = df.alleles.explode() == "A"
df.loc[idx[idx].index]

,id,alleles
0,1,"[A, C]"
1,2,"[G, A]"


In [5]:
# 3. variants that have allele containing a "A" base.
idx = df.alleles.explode().str.contains("A")
df.loc[pd.unique(idx[idx].index)] # call unique to remove duplicate rows (since "A" can appear in more than one allele in a row)

,id,alleles
0,1,"[A, C]"
1,2,"[G, A]"
3,4,"[G, GA, GAC]"


In [6]:
# 4. variants that have no alt alleles.
idx = df.alleles.explode().index.drop_duplicates(keep=False)
df.loc[idx]

,id,alleles
2,3,[T]


## Parquet and DuckDB

We can also use another query engine to run the queries by saving in Parquet format. I used DuckDB to get the same results.

In [7]:
# Save as a Parquet file (needs pyarrow package)
df.to_parquet("alleles.parquet")

In [8]:
# Equivalent queries using duckdb
# select * from 'alleles.parquet' where id > 1;
# select * from 'alleles.parquet' where list_contains(alleles, 'A');
# select * from 'alleles.parquet' where list_any_value(list_transform(alleles, x -> CONTAINS(x, 'A') OR NULL));
# select * from 'alleles.parquet' where len(alleles) = 1;

## Sgkit, Xarray, and Zarr

For completeness, let's finally look at how to run these queries using sgkit backed by Xarray and Zarr. Note that we are not using ragged string arrays in this case - the `variant_allele` array is just padded to have a default width of 4 (alleles).

The queries are all possible, but the ones for `variant_allele` are not very intuitive (but there may of course be a better way of doing them!).

In [9]:
import numpy as np
import xarray as xr
import sgkit as sg
from sgkit.io.vcf import read_vcf

In [10]:
ds = read_vcf("sgkit/tests/io/vcf/data/sample.vcf.gz")
ds = ds[["variant_position", "variant_id", "variant_allele"]]
ds.load()

<xarray.Dataset>
Dimensions:           (variants: 9, alleles: 4)
Dimensions without coordinates: variants, alleles
Data variables:
    variant_position  (variants) int32 111 112 14370 ... 1234567 1235237 10
    variant_id        (variants) object '.' '.' 'rs6054257' ... '.' 'rsTest'
    variant_allele    (variants, alleles) object 'A' 'C' '' '' ... 'A' 'ATG' 'C'
Attributes:
    contigs:               ['19', '20', 'X']
    filters:               ['PASS', 's50', 'q10']
    max_alt_alleles_seen:  3
    source:                sgkit-0.6.1.dev2+gcc728043
    vcf_header:            ##fileformat=VCFv4.0\n##FILTER=<ID=PASS,Descriptio...
    vcf_zarr_version:      0.2

In [11]:
# 1. variants that have a certain position.
ds.isel(variants=(ds.variant_position > 1000000))

<xarray.Dataset>
Dimensions:           (variants: 4, alleles: 4)
Dimensions without coordinates: variants, alleles
Data variables:
    variant_position  (variants) int32 1110696 1230237 1234567 1235237
    variant_id        (variants) object 'rs6040355' '.' 'microsat1' '.'
    variant_allele    (variants, alleles) object 'A' 'G' 'T' '' ... 'T' '' '' ''
Attributes:
    contigs:               ['19', '20', 'X']
    filters:               ['PASS', 's50', 'q10']
    max_alt_alleles_seen:  3
    source:                sgkit-0.6.1.dev2+gcc728043
    vcf_header:            ##fileformat=VCFv4.0\n##FILTER=<ID=PASS,Descriptio...
    vcf_zarr_version:      0.2

In [12]:
# 2. variants that have an "A" allele.
import functools

def string_equals(x, *, val):
    return np.any(np.vectorize(lambda s: s == val)(x), axis=-1)

func = functools.partial(string_equals, val="A")

da = xr.apply_ufunc(func, ds.variant_allele, input_core_dims=[["alleles"]], dask="allowed")
ds.sel(variants=(da)).variant_allele

<xarray.DataArray 'variant_allele' (variants: 6, alleles: 4)>
array([['A', 'C', '', ''],
       ['A', 'G', '', ''],
       ['G', 'A', '', ''],
       ['T', 'A', '', ''],
       ['A', 'G', 'T', ''],
       ['AC', 'A', 'ATG', 'C']], dtype=object)
Dimensions without coordinates: variants, alleles
Attributes:
    comment:  The possible alleles for the variant.

In [13]:
# 3. variants that have allele containing a "T" base.
def string_contains(x, *, val):
    return np.any(np.vectorize(lambda s: val in s)(x), axis=-1)

func = functools.partial(string_contains, val="T")

da = xr.apply_ufunc(func, ds.variant_allele, input_core_dims=[["alleles"]], dask="allowed")
ds.sel(variants=(da)).variant_allele

<xarray.DataArray 'variant_allele' (variants: 5, alleles: 4)>
array([['T', 'A', '', ''],
       ['A', 'G', 'T', ''],
       ['T', '', '', ''],
       ['T', '', '', ''],
       ['AC', 'A', 'ATG', 'C']], dtype=object)
Dimensions without coordinates: variants, alleles
Attributes:
    comment:  The possible alleles for the variant.

In [14]:
# 4. variants that have no alt alleles.
def no_alt(x):
    return np.sum(np.vectorize(lambda s: s != "")(x), axis=-1) == 1

da = xr.apply_ufunc(no_alt, ds.variant_allele, input_core_dims=[["alleles"]], dask="allowed")
ds.sel(variants=(da)).variant_allele

<xarray.DataArray 'variant_allele' (variants: 2, alleles: 4)>
array([['T', '', '', ''],
       ['T', '', '', '']], dtype=object)
Dimensions without coordinates: variants, alleles
Attributes:
    comment:  The possible alleles for the variant.